In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from util import getData
%matplotlib inline

In [2]:
X_train, X_test, y_train, y_test = getData()

In [11]:
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [8]:
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.layers.core import Activation
from keras.models import Model

In [4]:
N = X_train.shape[0]
D = X_train.shape[1]
M1 = 2000
M2 = 1000
M3 = 500
K = 7

### stacked autoencoder実装。(無駄があるので、もう少しいい書き方がありそう)

In [43]:
def autoencoder_each_layer(layer, next_layer, input_val):
    """
    各層のautoencoder計算用メソッド。別のmodelを定義して計算しているが、しなくてもできそう。
    SequentialじゃないFunction API使えばできそう。
    """
    model = Sequential()
    model.add(layer)
    model.add(Activation("relu"))
    model.add(next_layer)
    model.add(Activation("sigmoid"))
    batch_size = 256
    nb_epoch = 50
    model.compile(loss="binary_crossentropy", optimizer="adadelta")
    model.fit(input_val, input_val, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, shuffle=True)
    return model.get_weights()[0], model.get_weights()[1]

In [44]:
def predict_input_value(model, x):
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model.predict(x)

In [50]:
weights = []
dims = [D, M1, M2, M3, K]
input_value = X_train
stacked_model = Sequential()
for i in range(len(dims) - 1):
    layer = Dense(dims[i + 1], input_shape=(dims[i + 0],))
    next_layer = Dense(dims[i + 0])
    w, b = autoencoder_each_layer(layer, next_layer, input_value)
    weights.append(w)
    weights.append(b)
    if i == 0:
        stacked_model.add(Dense(dims[i + 1], input_shape=(dims[i + 0],), weights=np.array([w, b])))
    else:
        stacked_model.add(Dense(dims[i + 1], weights=np.array([w, b])))
    input_value = predict_input_value(stacked_model, X_train)
    if i == len(dims) - 1:
        stacked_model.add(Activation("softmax"))
    else:
        stacked_model.add(Activation("relu"))
    

Epoch 1/1
32633/32633 [==============================] - 44s - loss: 0.6967    
Epoch 1/1
32633/32633 [==============================] - 21s - loss: -0.3762    
Epoch 1/1
32633/32633 [==============================] - 7s - loss: -0.7088     
Epoch 1/1
32633/32633 [==============================] - 1s - loss: 0.4855     


In [ ]:
batch_size = 1000
nb_epoch = 100
stacked_model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
stacked_model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_test, y_test),verbose=1, shuffle=True)